# Session I: Regression

In [ ]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import math

#plt.rc('text', usetex=True)

Data are provided as equally spaced points in the interval $[0,1]$, from the function $\sin{2\pi x}$, adding a small level of random noise with Gaussian distribution

In [ ]:
N = 10                                                         # Number of datapoints
x_draw = np.linspace(0,1,100)                                  # x-points to draw the sine function
x = np.linspace(0,1,N)                                         # Equally-spaced points in the interval
t = np.sin(2*math.pi*x) + 0.3*np.random.normal(size=(N,))      # Datapoints
t_test = np.sin(2*math.pi*x) + 0.3*np.random.normal(size=(N,)) # Datapoints for testing

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
ax.plot(x_draw,np.sin(2*math.pi*x_draw),'g')

plt.show()

We try to fit the data using a polynomial function of order $M$, minimizing the mean-squared error with respect to the provided data

$$ y(x,\mathbf{w}) = w_0 + w_1x + \dotsc + w_Mx^M = \sum^M_{j=0} w_jx^j $$

$$ E(\mathbf{w}) = \frac{1}{2} \sum^N_{n=1} \{ y(x_n, \mathbf{w}) - t_n \}^2 $$


In [ ]:
poly_coeff = lambda x,M: np.array([x**i for i in range(M+1)])
poly_eval = lambda x,w: np.sum(w*poly_coeff(x, len(w)-1))

def OLS(M):
    X = np.zeros((N,M+1), dtype='float')
    for i_sample in range(N):
        X[i_sample] = poly_coeff(x[i_sample],M)
    
    w = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T,X)),X.T),t)
    return w

In [ ]:
M = 4                                                        # Degree of the polynomial

w_OLS = OLS(M)
y_OLS = np.array([poly_eval(x_draw[i], w_OLS) for i in range(len(x_draw))])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
ax.plot(x_draw,np.sin(2*math.pi*x_draw),'g')
ax.plot(x_draw,y_OLS,'r')

plt.show()

#print(w_OLS)

In [ ]:
RMS = lambda w,x,t: np.sum(np.array([poly_eval(x[i], w)-t[i] for i in range(len(x))])**2)/len(x)

W = list()
Y_OLS = np.ndarray((N,len(x_draw)), dtype='float')
E_RMS_training = np.ndarray((N,), dtype='float')
E_RMS_test = np.ndarray((N,), dtype='float')
for i_M in range(N):
    w_OLS = OLS(i_M)
    W.append(w_OLS)
    Y_OLS[i_M] = np.array([poly_eval(x_draw[i], w_OLS) for i in range(len(x_draw))])
    E_RMS_training[i_M] = RMS(w_OLS,x,t)
    E_RMS_test[i_M] = RMS(w_OLS,x,t_test)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
ax.plot(x,t_test,'^')
ax.plot(x_draw,np.sin(2*math.pi*x_draw),'g')
    
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(E_RMS_training,'-o')
ax.plot(E_RMS_test,'-o')        

plt.show()

In [ ]:
for i in (0,1,3,9):    
    print(W[i])

## $L_2$ Regularization

In order to reduce the magnitude of the model parameters we can add *regularization*

$$ E(\mathbf{w}) = \frac{1}{2} \sum^N_{n=1} \{ y(x_n, \mathbf{w}) - t_n \}^2 + \frac{\lambda}{2} \| \mathbf{w} \|^2$$

In [ ]:
def OLS_wL2(M, loglamb):
    X = np.zeros((N,M+1), dtype='float')
    for i_sample in range(N):
        X[i_sample] = poly_coeff(x[i_sample],M)
    L2_matrix = np.exp(loglamb)*np.eye(M+1)
    L2_matrix[0,0] = 0
    w = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T,X)+L2_matrix),X.T),t)
    return w

In [ ]:
M = 1                                                        # Degree of the polynomial
loglamb = -10                                                # Log of the regularization parameter  

w_L2 = OLS_wL2(M, loglamb)
y_L2 = np.array([poly_eval(x_draw[i], w_L2) for i in range(len(x_draw))])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
ax.plot(x_draw,np.sin(2*math.pi*x_draw),'g')
ax.plot(x_draw,y_L2,'r')

plt.show()

In [ ]:
print(w_L2) # M = 9

In [ ]:
print(w_L2) # M = 5

# Things that I am skipping
* More complex linear models
* Multivariate models
* Bayesian approach
* *etc.*

# Stochastic gradient descent (SGD)

In [ ]:
M = 1

w1 = np.linspace(-3,3,100)
w2 = np.linspace(-3,3,100)
err_map = np.zeros((len(w1),len(w2)), dtype='float')
for i_1 in range(len(w1)):
    for i_2 in range(len(w2)):
        w_map = np.array([w1[i_1], w2[i_2]])
        err_map[i_1,i_2] = RMS(w_map,x,t)
        
fig = plt.figure()
ax = fig.add_subplot(111)        
ax.contour(w1,w2,err_map,40)

plt.show()

Given the error function $E(\mathbf{w})$ and an initial guess $\mathbf{w}^{(0)}$, the parameters of the model are updated iteratively

$$ \mathbf{w}^{(i+1)} = \mathbf{w}^{(i)} - \eta\frac{\partial E(\mathbf{w}^{(i)})}{\partial \mathbf{w}^{(i)}} \quad \text{for }i=1,2,\dotsc $$

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)        
ax.contour(w1,w2,err_map,40)

w_GD = np.random.normal(size=(M+1,))
ax.plot(w_GD[1],w_GD[0],'ro')
ax.plot(w_L2[1],w_L2[0],'r*')

plt.show()

For stochastic gradient descent, we consider the error function without regularization and we rewrite it as

$$ E(\mathbf{w}) = \sum^N_{n=1} E_n(\mathbf{w}) = \sum^N_{n=1} \frac{1}{2} \{ y(x_n, \mathbf{w}) - t_n \}^2 $$

then we approximate it with

$$ E(\mathbf{w}) \approx \sum^{N_b}_{n=1} E_n(\mathbf{w}) $$

where $N_b \ll N$.

In [ ]:
# Run again the regularized optimization with M = 5
M = 5

lr = 0.05
tol = 1e-5

w_SGD = np.random.normal(size=(M+1,)) # Initialization of the weight array
delta_W = np.ones((M+1,),dtype='float')
samples = range(N)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
n_iter = 0
err = list()
while (np.linalg.norm(delta_W)>tol and n_iter<5000):
### SGD Algorithm here
    # Randomize samples
    # loop over samples/batches
        # compute the error and append it to the err list
        
        # compute the gradient

        # update weights

    ax.plot(x_draw, np.array([poly_eval(x_draw[i], w_SGD) for i in range(len(x_draw))]))
    n_iter += 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,t,'o')
ax.plot(x_draw, np.array([poly_eval(x_draw[i], w_SGD) for i in range(len(x_draw))]))
ax.plot(x_draw,y_L2,'r')

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(err[::10])

plt.show()